<a name="top"></a><img src="images/chisel_1024.png" alt="Chisel logo" style="width:480px;" />

# Module 2.1: 你的第一个Chisel模块
**Prev: [Scala 简介](1_intro_to_scala.ipynb)**<br>
**Next: [组合逻辑](2.2_comb_logic.ipynb)**

## 目标
现在您已经熟悉了Scala，让我们开始开发一些硬件单元！ Chisel 代表 **C**onstructing **H**ardware **I**n a **S**cala **E**mbedded **L**anguage。 这意味着它是Scala中的DSL，使您可以在同一套代码中利用 Scala 和 Chisel 编程。 了解哪些代码属于 `Scala` 和哪些代码属于 `Chisel` 是很重要的，但是我们将在后面讨论。 现在让我们直接将 Chisel 和模块2中的代码当作优于Verilog的开发方法。这一章向你展示了 Chisel 模块和其测试框架。暂时了解它的要点。稍后您将看到更多示例。

## 设置
以下单元格将下载Chisel所需的依赖项。您将在以后的所有 Notebook 中看到它。 **请先运行此单元格**。

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

如上所述，导入 Chisel 时需要这些语句。**现在运行此单元格**，然后再运行下面的其他代码块。

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.iotesters.{ChiselFlatSpec, Driver, PeekPokeTester}

---
# 你的第一个模块
本节将介绍您的第一个硬件模块，一个测试用例以及运行它的方法。它将包含许多您不了解的内容，不过这没关系。我们希望您大致掌握，因此您可以不断返回到这章的完整且有效的示例，以巩固您所学的内容。
<span style="color:blue">**例子：模块**</span><br>
像Verilog一样，我们可以在Chisel中声明模块定义。 以下示例是 Chisel 模块 `Passthrough`，具有一个4位输入 `in` 和一个4位输出 `out`。该模块组合连接`in` 和 `out`，因此 `in` 直接驱动 `out`。

In [ ]:
// Chisel代码：声明一个模块
class Passthrough extends Module {
  val io = IO(new Bundle {
    val in = Input(UInt(4.W))
    val out = Output(UInt(4.W))
  })
  io.out := io.in
}

以下有更多！下面对每行解释了针对需要生成的硬件来进行思考。
```scala
class Passthrough extends Module {
```
我们定义了一个叫做 `Passthrough` 的新模块. `Module` 是 Chisel 内置的类，所有的硬件模块必须从此继承。

```scala 
val io = IO(...)
```
我们定义我们的输入输出端口用特殊变量 `io`. 他必须被命名为 `io` 而且是一个 `IO` 的对象, 这需要利用 `IO(_instantiated_bundle_)` 来生成.

```scala
new Bundle {
    val in = Input(...)
    val out = Output(...)
}
```
我们声明了一个新的硬件结构体类型（Bundle）：包含 `in` 信号和 `out` 信号，它们分别具有 `Input` 和 `Output` 的方向。
```scala
UInt(4.W)
```
我们声明信号的硬件类型。 在这种情况下，它是宽度为4的无符号整数。

```scala
io.out := io.in
```

我们将输入端口连接到输出端口，例如 `io.in` 驱动 `io.out`。 请注意，`:=` 是 ***Chisel*** 操作符，它表示右侧信号驱动左侧信号。 它是有向操作符。

关于硬件构造语言（HCL）的优雅之处在于，我们可以将基础编程语言用作脚本语言。例如，在声明我们的Chisel模块之后，我们然后使用Scala调用Chisel编译器，将Chisel的 `Passthrough` 转换为Verilog `Passthrough`。 此过程称为***elaboration***。

In [ ]:
// Scala 代码： 对 Chisel 代码转换成 Verilog 来进行 Elaborate 
// 不用担心看不懂其代码，这是非常复杂的 Scala
println(getVerilog(new Passthrough))

请注意，我们模块的名称是 `cmd<#>WrapperHelperPassthrough`，这是在Jupyter中运行本教程的产物。在您的常规代码中，其名称应该只是 `Passthrough`。这是重要的一件事：尽管Chisel尽其所能保留模块和其他硬件组件的名称，但有时却无法做到。

<span style="color:blue">**例子：一个模块生成器**</span><br>
如果将对 Scala 的了解应用于本示例，则可以看到 Chisel 模块是作为 Scala 类实现的。与其他 Scala 类一样，我们可以使 Chisel 模块采用一些构造参数。在这种情况下，我们创建一个新的类 `PassthroughGenerator` ，该类将接受一个整型参数`width`，该宽度定义了其输入和输出端口的宽度：

In [ ]:
// Chisel 代码, 通过传入参数设置端口的位宽
class PassthroughGenerator(width: Int) extends Module { 
  val io = IO(new Bundle {
    val in = Input(UInt(width.W))
    val out = Output(UInt(width.W))
  })
  io.out := io.in
}

// 我们现在来生成一些不同位宽的模块
println(getVerilog(new PassthroughGenerator(10)))
println(getVerilog(new PassthroughGenerator(20)))

请注意，根据分配给 `width` 参数的值，可以对生成的Verilog的输入/输出使用不同的位宽。让我们研究一下它是如何工作的。因为 Chisel 模块是普通的 Scala 类，所以我们可以使用 Scala 的类构造函数的功能来参数化设计的细节。

您可能会注意到，此参数化是来自于 *Scala* 而非 *Chisel*； Chisel没有其他用于参数化的API，但是设计人员可以简单地利用 Scala 的特性对他/她的设计进行参数化。

由于 `PassthroughGenerator` 不再描述单个模块，而是描述了由 `width` 参数化的一系列模块，因此我们将 `Passthrough` 称为 ***生成器***。

---
# 测试您的硬件

没有测试的硬件模块或生成器都不是完善的。Chisel 具有内置的测试功能，您将在本训练营中进行探索。以下示例是 Chisel 测试工具，它是将值传递到 `Passthrough` 输入端口 `in` 的实例，并检查在输出端口 `out` 上是否有相同的值。

<span style="color:blue">**例子: 测试**</span><br>
这里有一些进阶的Scala代码。 但是，除了 `poke` 和 `expect` 命令之外，您不需要了解其他任何内容。您可以将其余代码视为编写这些简单测试的简单样板。

In [ ]:
// Scala代码：调用 Driver 实例化 Passthrough 和 PeekPokeTester 以执行测试。
// 不用担心理解此代码； 这是非常负责的Scala。
// 可以将其视为运行 Chisel 测试的样板.
val testResult = Driver(() => new Passthrough()) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 0)     // Set our input to value 0
    expect(c.io.out, 0)  // Assert that the output correctly has 0
    poke(c.io.in, 1)     // Set our input to value 1
    expect(c.io.out, 1)  // Assert that the output correctly has 1
    poke(c.io.in, 2)     // Set our input to value 2
    expect(c.io.out, 2)  // Assert that the output correctly has 2
  }
}
assert(testResult)   // Scala 代码: 如果 testResult == false, 那么将抛出异常
println("成功!!") // Scala 代码: 如果我们的测试执行至此，那我们测试成功运行了!

What's going on? The test accepts a `Passthrough` module, assigns values to the module's inputs, and checks its outputs. To set an input, we call `poke`. To check an output, we call `expect`. If we don't want to compare the output to an expected value (no assertion), we can `peek` the output instead.
发生了什么？该测试接受一个 `Passthrough` 模块，对模块的输入给值，并检查其输出。要设置输入，我们称为 `poke`。为了检查输出，我们调用 `expect`。如果我们不想将输出与期望值进行比较（无断言），则可以 `peek` 输出。

如果所有的 `expect` 语句都为 true，则我们的样板代码将返回 true（请参见 `testResult` ）。

<span style="color:red">**练习: 写你自己的测试**</span><br>
编写并执行两个测试，一个测试宽度为10的 `PassthroughGenerator`，第二个测试宽度为20。检查每个值至少两个：零和指定宽度支持的最大值。请注意，三重问号在 Scala 中具有特殊含义。 您会在这些训练营练习中经常看到它。运行带有 `???` 的代码将产生 `NotImplementedError`。 请用您的测试代码替换 `???` 完成测试。

In [ ]:
val test10result = ???

val test20result = ???

assert((test10result == true) && (test20result == true))
println("成功!!") // Scala 代码: 如果我们的测试执行至此，那我们测试成功运行了!

<div id="container"><section id="accordion"><div>
<input type="checkbox" id="check-1" />
<label for="check-1"><strong>解答</strong> (单击显示)</label>
<article>
<pre style="background-color:#f7f7f7">
val test10result = Driver(() => new PassthroughGenerator(10)) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 0)
    expect(c.io.out, 0)
    poke(c.io.in, 1023)
    expect(c.io.out, 1023)
  }
}

val test20result = Driver(() => new PassthroughGenerator(20)) {
  c => new PeekPokeTester(c) {
    poke(c.io.in, 0)
    expect(c.io.out, 0)
    poke(c.io.in, 1048575)
    expect(c.io.out, 1048575)
  }
}

</pre></article></div></section></div>

---
# 查看生成的 Verilog/FIRRTL

如果您在理解生成的硬件时遇到麻烦，并且对阅读结构化的 Verilog 或 FIRRTL（类似于可综合 Verilog 子集的 Chisel 的 IR）感到满意，则可以通过这个尝试查看生成的 Verilog 以获取 Chisel 生成的结果。
这是一个查看Verilog（你已经见过）和 Firrtl 的例子

In [ ]:
// 查看 Verilog 用来 Debug
println(getVerilog(new Passthrough))

In [ ]:
// 查看 Firrtl 用来 Debug
println(getFirrtl(new Passthrough))

---
# 你搞定了!

[回到开头](#top)

## <span style="color:red"> 附录: 关于使用 "printf" 调试</span>
[Debugging with print statements](https://stackoverflow.com/a/189570) 并非总是最佳的调试方法，但通常是第一步的尝试，它可以很容易地了解当某些事情无法按预期方式工作时发生了什么。

因为 Chisel 生成器是生成硬件的程序，所以有关打印生成器和电路状态还有一些额外的细微之处。
重要的是要记住您的打印语句何时执行以及正在打印什么。
您可能要打印的三种常见方案有一些重要区别：
* 电路生成时 Chisel 生成器的输出
* 仿真时电路的输出
* 测试时测试程序的输出

`println` 是内置的Scala函数，可打印字符串到控制台. 它 **不能** 用于电路仿真期间进行打印，因为生成的电路是FIRRTL或Verilog，而非Scala。

以下代码块显示了不同的打印方式。

In [ ]:
class PrintingModule extends Module {
    val io = IO(new Bundle {
        val in = Input(UInt(4.W))
        val out = Output(UInt(4.W))
    })
    io.out := io.in

    printf("仿真时输出: 输入是 %d\n", io.in)
    // chisel 的 printf 有其自己的字符串格式化工具
    printf(p"仿真时输出: IO 是 $io\n")

    println(s"电路生成时输出: 输入是 ${io.in}")
}

class PrintingModuleTester(c: PrintingModule) extends PeekPokeTester(c) {
    poke(c.io.in, 3)
    step(5) // 电路将打印出来
    
    println(s"测试时输出: 输入是 ${peek(c.io.in)}")
}
chisel3.iotesters.Driver( () => new PrintingModule ) { c => new PrintingModuleTester(c) }